In [1]:
### 載入套件
import requests
from bs4 import BeautifulSoup as bs 

In [2]:
### 取得 首頁 source code
url = "https://www.nownews.com/cat/column/"

headers= {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url,headers=headers)
soup = bs(res.text,"lxml")

In [3]:
### 抓取 1 ~ 10 篇文章連結
links = []

# 抓取 1~5 篇
for ele in soup.select("div.sliderBlk a"):
    url = ele["href"]
    links.append(url)
    
# 抓取 6~10 篇
for ele in soup.select("div.leftCol div.listBlk a"):
    links.append(ele["href"])
    
for url in links:
    print(url)

https://www.nownews.com/news/5429052
https://www.nownews.com/news/5428984
https://www.nownews.com/news/5428967
https://www.nownews.com/news/5428959
https://www.nownews.com/news/5429139
https://www.nownews.com/news/5428961
https://www.nownews.com/news/5428884
https://www.nownews.com/news/5429130
https://www.nownews.com/news/5429085
https://www.nownews.com/news/5429128


In [4]:
### 抓取本文資料 -> 收集成一個 function 
def parsing_content_data(content_soup):
    
    # 標題
    title = content_soup.select("div.titleBlk h1")[0].text

    # 分類
    # \n -> 換行符號
    category = content_soup.select("div.breadCrumbBlk")[0].text
    category = category.strip().replace("\n\n"," > ")

    # 時間
    # time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    # 時間格式更新
    raw_time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    time = raw_time.split("｜")[0]

    # 內容
    article = content_soup.select("div.leftCol article")[0]

    # 拔除 div.ad-blk1 , ul.related 標籤
    if len(article.select("div.ad-blk1"))>0:
        article.select("div.ad-blk1")[0].extract()
    if len(article.select("ul.related")) >0:
        article.select("ul.related")[0].extract()

    content = article.text.strip()
    
    return {
        "title" : title,
        "category" : category,
        "time"  : time,
        "content" : content
    }


def get_source_code(url):
    headers= {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }
    res = requests.get(url,headers=headers)
    soup = bs(res.text,"lxml")
    
    return soup



url3 = links[8]
print(url3)
soup3 = get_source_code(url3)
parsing_content_data(soup3)

https://www.nownews.com/news/5429085


{'title': '快訊／國旅券最後一抽！中籤號碼出爐\u30006組幸運號碼看這',
 'category': 'NOWnews 今日新聞 > 生活',
 'time': '2021-11-02 10:34:07',
 'content': '交通部觀光局配合振興五倍券，加碼推出240萬份面額1000元的國旅券，分為四週抽籤，前三週已抽出181萬份，今（2）日迎來最後一週抽籤日，在上午10時30分已抽出最後60萬名幸運兒，分別為身分證末2碼的「32」、「02」、「87」、「93」、「82」、「17」，總計有60萬227人中籤。\n\r\n\t\t\t\t            \t觀光局規劃推出240萬份的千元國旅券，在前三週已進行抽籤，抽出181萬名幸運兒，並在昨日起開放至抽籤民眾到合作的旅行社業者、遊樂園、旅宿業等管道使用使用，有效期限為明年的4月30日。前三週的中籤號碼分別為第一週的身分證末2碼為21、32、67、97、98及末3碼為410；第二週的身分證末2碼04、29、40、71、87；第三週的身分證末2碼044、34、09、55、35及末3碼為041。中籤者到「國旅券平台」輸入「手機號碼」+「身分證」或「居留證」末4碼登入後，即可領取國旅券QR Code代碼。（編輯：林莞茜）'}

In [5]:
### 抓取 1 ~ 10 篇文章本文
dataList = []
for url in links:
    content_soup = get_source_code(url)
    data = parsing_content_data(content_soup)
    dataList.append(data)
    
    print("{} is ok.".format(url))
    
print("Done.")

https://www.nownews.com/news/5429052 is ok.
https://www.nownews.com/news/5428984 is ok.
https://www.nownews.com/news/5428967 is ok.
https://www.nownews.com/news/5428959 is ok.
https://www.nownews.com/news/5429139 is ok.
https://www.nownews.com/news/5428961 is ok.
https://www.nownews.com/news/5428884 is ok.
https://www.nownews.com/news/5429130 is ok.
https://www.nownews.com/news/5429085 is ok.
https://www.nownews.com/news/5429128 is ok.
Done.


In [6]:
### 資料輸出成檔案 (資料落地)
for data in dataList:

    file_name = "sample/{}.txt".format(data["title"])
    with open(file_name,"w",encoding="utf8") as out_file:
        
        record = ""
        
        # 資料全部讀取完 , 再一次寫出檔案
        for key in data:
            record += "{}:{}\n".format(key , data[key])
            
        out_file.write(record)

        # 每讀一段資料 , 就寫出檔案一次 (不佳)
        # for key in data:
        #     msg = "{}:{}\n".format(key , data[key])
        #     out_file.write(msg)

In [7]:
### 透過 API 抓 JSON 資料
# 1. 抓取 txtPageNo , 取得 1st 參數
pid = soup.select("input#txtPageNo")[0]["value"]
print(pid)

# 2. 向 API 發 requests , 取得文章列表資料
api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}".format(pid)
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(api,headers=headers)

# response 用文字格式輸出
# print(res.text)

# response 用 json 格式輸出
data = res.json()

links2 = []

for ele in data["data"]["newsList"]:
    print(ele["postTitle"])
    # print(ele["postUrl"])
    
    url = "https://www.nownews.com" + ele["postUrl"]
    print(url)
    links2.append(url)

    print("-"*80)
    
# ele 停留在最後一筆資料
pid = ele["id"]


# 3. 再透過 API 的資料 , 取得下一次的 requests 參數
for i in range(1,4):
    api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}".format(pid)
    headers = {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }

    res = requests.get(api,headers=headers)
    
    data = res.json()
    
    for ele in data["data"]["newsList"]:
        url = "https://www.nownews.com" + ele["postUrl"]
        links2.append(url)
        
    pid = ele["id"]
    
print("Done.")

5428751
日職／好消息！王柏融有機會留日本　火腿主動交涉
https://www.nownews.com/news/column/5429011
--------------------------------------------------------------------------------
誇張！知名飯店「女裸湯區闖入男員工」　櫃檯回應引眾怒
https://www.nownews.com/news/column/5429007
--------------------------------------------------------------------------------
台灣缺完整移民政策　林麗蟬：關鍵在「新住民」定義模糊
https://www.nownews.com/news/column/5428993
--------------------------------------------------------------------------------
強生：若COP26在氣候議題上失敗　眾怒將無法控制
https://www.nownews.com/news/column/5428846
--------------------------------------------------------------------------------
聯合國秘書長稱人類自掘墳墓　籲氣候峰會有所行動
https://www.nownews.com/news/column/5428847
--------------------------------------------------------------------------------
赤裸裸的歧視！本勞、移工「同工不同酬」亟需改善
https://www.nownews.com/news/column/5428992
--------------------------------------------------------------------------------
格陵蘭島冰層10年內融冰3.5兆噸　全球洪災風險升高
https://www.nownews.com/news/column/5428783
-------------------

In [15]:
### GOGOGO !!!
### 透過 API 抓 JSON 資料
### 4. 整理版 (未教學完成)
# 1. 抓取 txtPageNo , 取得 1st 參數
pid = soup.select("input#txtPageNo")[0]["value"]
print(pid)


api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}"
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

# 2. 向 API 發 requests , 取得文章列表資料
# 3. 再透過 API 的資料 , 取得下一次的 requests 參數
for i in range(1,4):
    url = api.format(pid)

    res = requests.get(url,headers=headers)
    
    data = res.json()
    
    for ele in data["data"]["newsList"]:
        url = "https://www.nownews.com" + ele["postUrl"]
        links2.append(url)
        
    pid = ele["id"]

5428751


In [ ]:
################################################################

In [ ]:
################################################################

In [ ]:
################################################################

In [18]:
### 整理程式
import time
import requests
from bs4 import BeautifulSoup as bs 

### function 
def parsing_content_data(content_soup):
    
    # 標題
    title = content_soup.select("div.titleBlk h1")[0].text

    # 分類
    # \n -> 換行符號
    category = content_soup.select("div.breadCrumbBlk")[0].text
    category = category.strip().replace("\n\n"," > ")

    # 時間
    # time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    # 時間格式更新
    raw_time = content_soup.select("div.titleBlk p.time")[0].text.strip()
    time = raw_time.split("｜")[0]

    # 內容
    article = content_soup.select("div.leftCol article")[0]

    # 拔除 div.ad-blk1 , ul.related 標籤
    if len(article.select("div.ad-blk1"))>0:
        article.select("div.ad-blk1")[0].extract()
    if len(article.select("ul.related")) >0:
        article.select("ul.related")[0].extract()

    content = article.text.strip()
    
    return {
        "title" : title,
        "category" : category,
        "time"  : time,
        "content" : content
    }


def get_source_code(url):
    headers= {
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }
    res = requests.get(url,headers=headers)
    soup = bs(res.text,"lxml")
    
    return soup

### 主程式

# 取得主頁 source code
url = "https://www.nownews.com/cat/column/"

headers= {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url,headers=headers)
soup = bs(res.text,"lxml")

print("* 主頁抓取完成！")


# 抓取 1 ~ 10 篇文章連結
links = []

for ele in soup.select("div.sliderBlk a"):
    url = ele["href"]
    links.append(url)
    
for ele in soup.select("div.leftCol div.listBlk a"):
    links.append(ele["href"])
    
print("* 1 ~ 10 篇文章連結抓取完成！")


# 透過 API 抓取更多文章 (~50篇)
pid = soup.select("input#txtPageNo")[0]["value"]
print(pid)

api = "https://www.nownews.com/nn-client/api/v1/cat/column/?pid={}"

for i in range(1,2):
    url = api.format(pid)

    res = requests.get(url,headers=headers)
    
    data = res.json()
    
    for ele in data["data"]["newsList"]:
        more_link = "https://www.nownews.com" + ele["postUrl"]
        links.append(more_link)
        
    pid = ele["id"]
    
    time.sleep(0.5)
    
print("* 更多文章抓取完成！")


# 抓取文章本文
dataList = []
for url in links:
    content_soup = get_source_code(url)
    
    time.sleep(0.7)
    data = parsing_content_data(content_soup)
    dataList.append(data)
    
    print("{} is ok.".format(url))
    
    
print("* 文章本文抓取完成！")


### 資料輸出成檔案 (資料落地)
for data in dataList:
    file_name = "sample/{}.txt".format(data["title"])
    with open(file_name,"w",encoding="utf8") as out_file:
        
        record = ""
        for key in data:
            record += "{}:{}\n".format(key , data[key])
            
        out_file.write(record)

    
print("* 資料輸出成檔案完成！")
print("Done.")

* 主頁抓取完成！
* 1 ~ 10 篇文章連結抓取完成！
5429053
* 更多文章抓取完成！
https://www.nownews.com/news/5429186 is ok.
https://www.nownews.com/news/5429186 is ok.
https://www.nownews.com/news/5429185 is ok.
https://www.nownews.com/news/5429185 is ok.
https://www.nownews.com/news/5429148 is ok.
https://www.nownews.com/news/5429148 is ok.
https://www.nownews.com/news/5429180 is ok.
https://www.nownews.com/news/5429180 is ok.
https://www.nownews.com/news/5429052 is ok.
https://www.nownews.com/news/5429052 is ok.
https://www.nownews.com/news/column/5428963 is ok.
https://www.nownews.com/news/column/5428751 is ok.
https://www.nownews.com/news/column/5428830 is ok.
https://www.nownews.com/news/column/5428878 is ok.
https://www.nownews.com/news/column/5428731 is ok.
https://www.nownews.com/news/column/5429011 is ok.
https://www.nownews.com/news/column/5429007 is ok.
https://www.nownews.com/news/column/5428993 is ok.
https://www.nownews.com/news/column/5428846 is ok.
https://www.nownews.com/news/column/5428847 is ok.


In [ ]:
################################################################

In [ ]:
################################################################

In [ ]:
################################################################

In [12]:
### 台彩大樂透
import requests 
from bs4 import BeautifulSoup as bs
url = "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx"
headers = {
    "Host"      : "www.taiwanlottery.com.tw",
    "Origin"    : "http://www.taiwanlottery.com.tw",
    "Referer"   : "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}


payload = {
    "__VIEWSTATE": "EUlgvHEyVjB+aEiAawTMHIXFmvbgQqtCl2UDUYDxX33j/Z4aPcjOpf04xYoxJWSiXTVIfc4h9Y2PdMK20aDmuhZlUq4gyRLiZJBRw+Dc3UCZ6n5BP5NQNbBvHeFmtxgUO0+yjn7nu4yhM4D2IscTZEEg+/Zpkm09YAWg9vwybfd92Bqek4XbZWyTHe6dm8PowbyazQKV4SNcrgruHssctmOvo6p42J/lTOA/DL7IrRum8p4zYYiKXnpYABzbRg0xqd4UveEDH2Ariw1MDwrAT5R+9EJiMlYzEVhLEZaEXiNuk7RcUlGWHbPJ8loaHjF82+gGtg41YdBv0EulGa/RrKXkHutNFcHnZX4DtIvY2IR51nAgdJBZaa7rduvkapDv6igQ1nzbXIRjlHyo2tXf6qulPXLCM5jKTbsxu8H12g4bAd1pxecFlFxhLLrwffy/70eg0sWxIQ6VZEf7irF0fPOjlqvnXxEoPtPWTMtmsi5aYx5WGLAh6UVhe6NydUFqYiFyJl84ub8wsY++vzfm7WpubamdPifOsSimz8i2cQR+NFc3iaszVug74sgNKFpLjYNcp3wUyYGbEyJ82p2Ro8R1/opb6QCDogYepKMlqYtfFPloORuV3G1DxIUyrBlTL8GHQBJq+mflOMp0YHLwv9rKPZoYws5cOvn4MLg25s2mERyECB9gR7QkwjKPysoc/7emLU0Krmwymg/JpjLkIEmmmvozwvyUMkpV0qpe2IP7OEjsP6bjsdz4+a+n/yd5ErtvCGWxKQUTx4oRCp23XWDcawybFn91lBOuqYiZHJg0gbfGHslxmN/VlsyCDtliX5CEGHDhA5d89wtEu/Tw9uglydztkzP2RUckALekyRn2L4pN35MKEhUQO+PncdalFxyS6EwgwByhyTZR2t84lt2lpqV8vcZBJ0e/DbrSk20hDLZEwK2egvuN/BiNDG6iaUF0uj0geO+UBaIMRm6HPgLz2V+UqEXYMW8MsUcceKGjaBfLbF+6HJHf+b9UqKO9MAGBT7F4a9XSmM5+S11vJsSRpc63QhmtDLv8ix2xnp08XeKR0LvyIpVwakIQUMLWU77Af1s02RIiNbhg1KKwOJt2DTn2acsESObxM0Js8n2oCAUC+2kLv4WjxNTUIfuy1x2c13ZL/wo+a9Mnj4Bq69n4Yluqgpr/p+WUx3V00qLWx9xUpehjOditgbfspJ036ABWrJV9XAIpO95KWQdyfMMIZ0W5Bso384Ko8Fe+0TZ6GtJkA5BJtpEr3X4SUFQO6l9LFLjybN2dos9FRBvAG7vq7zORvpKQs+ycISiwPAjbDAnPiVsHFyq/xhsV46/R0mWzikMXl2sQbsBA8jbTdWkPqpNKMqTCy0NGGtanrY1YWRwoX9f7mGZlC/JUnQs/+WIDsIt1g4TXMdpZfH49hwiGwM7T+J/trf7bTLQiOmqvH505XVQ3FkhnEo8R7JH/7JX45ZsgtNAENV8PO/Pus+qnm1XY5Pth/5g1f73eBuhfVZ0e9fWAE25jUej91G7e8D3b98r4mJjo/sDUkxXjb9VbyzvI0t4f9uxhJzVKrHN7cWEhLUvSAd1POrE0YTximsQPNtJbBrNfh6VotdN+1GZIeJTOQFnRyR8UpRRtKf8f4G3FhvlL5hcUF59R4/OWGasOqJAQ4dlUebfX9LSKMqBevlJiLcZDMHZWUa7J/BMPxsoe637GeIOXrEvDML/wmYZ5LmEoT1vhA6FZTjdepAesK7pbgD+f0+Po2Rsw4yrsI6nLuQhTuN1sVwlqv3AAoTM3ah9QuozgMY5ROqQgUnwfkATzxHiJv3JKt3thkXyI08OZyuSpXh0Av/7gLnGn5zcuQF5YD7yWvdpcHYYJrEbjV4NpGruLgpRfqt2wzvfYnQTiIGi6QZNsqiSmR2b3KHaNrjf9by8Ov6uCrOhA9DyXaq+xLyA+YbI52vI0IhTdeOjVcRxnbS9cyMFpn1WlYmtGFwtCpj62gWWFyx/UA7Yd9uEBDLA40fzHx9uGQCnlu+3UTI2Xdi+nqA3osY5f/xj1a9JWp/yekifCVUx3ap7kWHje4J+ZjsXOfHaIoEtchLNf2uDZGTgX9m8gfR4YLSdP9k30G3uyoWKNIW+cT8OyHBmzbW6E+aDOGuVUDV0nORn9O5oMBEgYkzK+dUosH9g1f9t0d/F4opZYiLXpe5awNQwv/AHSEdaHljMNt3HGMCFP8D8KbGyF34SBCpFIht9D5RcEw1wjsUZKC0RAgi0wjpvxJYzlLsKaIuOge4yBOncUPFzm5q9dxvjAxAkNIkZjJPhqJYKsrlvDzt9PFKoy2J3OurgwVO0d0ew6PFK1xr7IQB6GPpVVBZ1CFv0VI8NdzWiFlSiHdmIkO+oNR7IRRY+PbdeLV3EwWLclCJtivyPsRhK40kwWoi6tf9y19LUHlD4LnvifQnLycYcRZbrbqKz7L/uV3EmYSgwPUgcTAOtQetuPP7HRGvO20GsMA/zh+wIlxDHil337O+U0meplaRg4RTUgUZXSzNT4dLvYMmFaKVgd2SADaxkaEyGZ2xZUIjqSYdCD4wI/aMzM73grHi75zEcy2UQ3y6+jYlDDZytwQ01Y9c7Aywt2HMUrxUId1ju8fy+ib5Y7E8UWzKM9wmAndq/ADkaqgLFkMniDiRWjTlEvRkcGeLfgMdpc6KtyHDG44dNqtsqz5EUkNZ3NyLhOHxbiBVdwr75iK7qozfbU7spzKmBUYTM34714TclzpFYtqRO+0jCRBspcpdU5z1kXSoe1039Xe/Zd4omUFghnZckiQA/qHEdETLvR1qLDCMDxBL8jSCOw3JvatXMjrXQC8odsEq2fBljoCqSsXQeYby088cPkgXiuIYDkx3BYGElrgk4BclccAZQJI+lI94WFvJ6KfDMUWDENHm43AWxEGvpBeJdi4pakm0z3F2eHz1QFBEBVaqMy6AMtmrRrkQ6m/5tLQFxlgjPNpl6oc288YXSOPtqBw3Y/u1cxdY5Y+SWtln9YNrFP54fZ6+iqJpSsLPg9qdNtHTDif9dfe4oToosC7bi0E09y9g+IWgdNuLddawshUnxNebfF9WIpNOL2Re6k3Pu2PfR+pEZmvUOD/nUq3qlZ9Rq/CqD98iBOWDM0Ix58PcqD/y2QGKKZf+VR2xFmI4UpoH3Bls2wnVYPV2RdE85WKkqBw5B400jf5PiNIBPmuF7L8EWl0BAgmLI7TrYx4v0RpWY8Hm2yWrnKGXjDPrglKq+PnqO3PnS4JrURCMNP1pUu4ssW2nCNnmSZ9natVqXEP0ExA6YXGpNLSCZR3Zff5SycF0IZeD0i7iGf3szyoRuIM11r7VmWsURBBNfOYg72Q7unjLh+KYIaVxzf7g2C5sH4dSqJh1RbYHH1JfX2CdrbIWhGVMdU60TXtDzDzA+H8FciS5LwgkzjJPfo1NT59U+uhdPJMtOQUMroKfaIUKhnTloadDHicnLvecZkZut8tl77IUkioAUcwm8XZnA0slYXxGvR0eNcHAz2Mu5LQabNF7kyGELgKnT4qmyUvlkPWdvH+GBsZIkMcmJ7/sBOFl/s8qqZ8+6hBv3evnKoLDjSc3PPwrz9cl0upZ4hhcXYPaAd+JVuqNdwqgOBD9Qia3iaumHzVUMh2xpW8479IQ4PgR4/cotX9y1LZ4/l/XEhHheLQjbBsrlpsq3nVJAm4fSbl1IqXo0TNp7sXWuB1aXXR5r08QBY9KTiILPhriNAIYCWnLnA80otMWDWahoXIxvFgCDxORh+Xn67SvEUiY8AOxZaCVEuc586GFI3JvCu6DoAZnwZF6c4VU5fI3CAts+KOXVxMqgVfSwxZXCyi6tr3b4Ks57TxEcub5d+b9/dgcfqLukU5OeDAZ7BgbYqv/rP1LvHcbPvKyB5mrn4hRVIQlnuOdkgbGqQV5ti5lRrmZGks1WFmOQsbIN4a3Bw0yvhB5jceLqfOoxdZE3q4yYq1I6UW9V/1SRT376gVL1Vuam0aj/nDMql/8zRUzQ6Rk6Z80POIYaBJdc/VYpp8H8YXfRynkFLw828vBjkhYiMzumbrHVmNuhLTw00Ki4PqLZ23szlP6BPkLl2HE0yf9xAcR4Shy+t4VulN/ZxKCpO7TM7eO3ztottoF5CSKI9qb/hmTWoDCGPs83ARpTJG6Z+nA6j700+a/knkYW2sdoTDX2YeVq6HUqW9PH4KDC4TidtDoBfR5jsG3sd7BeJk8tsDuZI2S3ixtulucAYH7UvrRuDSmYWqWxyuuvUbnwFlPsD8XoDSfyC6MLFf+nQ4XyyHbiZbpc/v4JhCRZmmMZABCRjS8N3C6h7hLQKCHs05IUwW4eXyZ8Vm5XftFZcwUI5CDUCWf02AzQv3XPmomr7umuZlHZCwSZ2Y+Wefctf+d2VsoHC26wv/eCHSZMJvzrLpkYSpqOc/xCNRqx8m6q3V0VCcKcNxgk5403QmGaEAXPTN089RUtvOF0aWvGAN8Xd0lDKI/tgUkkCWIYYoZEWxw9Csc8fII+fOAC3wq0ggbbaCqzs4VSGdqk6bTsOPs/6bEl4FytD+DzNWYCgcPsdvQlKPZ528fKp79ikUW7YyO61l8MJfffDhawSjpD0cNJcIRUIly4j6KZBqAe592Urqc+mSK3qHkyN/Y6QnuyJGzXMFcqtRorrNBMs6H7DilrxxVnV1beGpA5dx/8FI5L81aYrAMgOLHOHNp232+G32P9XeD5LIiM0CA9zw58rZwwtYoruggvdHP7Aj1/tIbg1pJUYwssKcQRVzwkurhDOwyAfBqs45tPbOyLbYqB8Fbf349Sc7Xf8kZUdvP1ZZGy2/G8O+tFNxqUxga+H4DAt8jk4EMHndzZRp8qQtJSyh3PPkJPlcIqdtj5F8ZJV1Y94lhWrjefG7BKvJlQmS7Cc6lxHIdvTlYny+f4ZmsREURNGPVB28lHKNX7v4+3V0jfm6xfPJhSr/MhyeGWF0ZRSXpENUULD+uJXy8cktJaQMd2zlJf+KPGQ11uQNLncNJ3XvNtAyUKR3ir9GgAXtq5YMLVWxJ6KKnws0hQZE0s3hzEi86Q9UF15mk7p7wTp49aTJDi9Pw6P5ftCo15McI17J4kZFEz6FbO7rgKR0bG5rKXhkzbZFpBjnFYfdtuUGpIvmJ+84+rSTt9dRSgc15SwVcCOpxiIUNF8gNVMz2SFz8F44yntqjcFSiTY4pXEWe4PnTCbdu8PuOBnTYJGgVOYfgZD0tdsN7U7+fnjxWMqCUEWuNW7fxJfrQMeRRWwnnC5kuiEsTTUMyM7rrYbDWL+UvbCfWqIIJlyM71ZXlh+p7j5/Ds3Z3ZYzSP1cn8YDh8yV+cCjnmCpyXcWh94PoDe2mZgjEMTA4L3zq4NUDiPepmtdKNBCFmQYPlcDVOfu0NMFhIOPttB/5Yh89fqqnxw2WEykzVh6JI0tMj520g+2oj7rK+WMYpkHeikcu7PxOlL2ozLQOY1Y2A96rH8lig1JzAK+5jUSETyshNPrYoNBfLLupcY5qg+xB0RzCuVPuGRaAkNOQHhCzAoGBVkypJzEJE8pt2xXgeDa4Dz5DbpBhBUW/8xqME5SM+LlZ/NbPSBqXI/OfMaMRWRwxLWipB9faaJwegZOw4tXl/d9evO9wYfptrBGEKv6p3puTvAoUW7dK606KX61FMX6jwXTN9H3vkqYJQx/IU3vg1vG4ztvMW9KIox2OuW/BHM0mC4n89HiTyJIw8DFOVGmrKZCHmdz7LqKIEBMz8Zdpnc6uM+UA42ZZZ/2g5o+/BOPuNy6p8wun23TCFRxuTMXGF+NVwi2rkoMo1h9CEaazLQ6GXJVdq7g54ctQPGQ5irhK1WtXLizavohzXAPFHeRGt9LpO9PrS+owGhJQgoX+9PtLPJnXajm3Ud2Vr1yQpglEA5UMLlol8V/MBAD7Mj4mq/MhSrrVRor1Ls1IeRS+Y1Nw0h+BSQj+cGPwd2hnWhmMkto/mLGK5S2kPMDK29oeL7evJ3nanYKOAr8SkdbsDLsWAHm8bkKh6NxEmX4IZGZidjRIv3cjmCaD/fR7N78QvAG8w211K264VQcMksjwrPE3Q+8FJD9u/sN6zliDgcCHKTx5dm6VvgIEDg3wkSLkBx+58ZwMY6rACdnbvwR8TCysgmkmddbZjGuWHEBRdNfXJrBsNKCm8xZztPjk1tIj+2jFwDjtKLoVEzvCly5c8eJVjgwNxydArvVV8T8f70LsGQfCOEGfjFv6QI1R3I0irKAmlPVtgKXgjaClStlqqErfTvXJ4ZeyNRlSwcuXK3hyqMQdFx6pxhRQNxwR3d6mjHtQYav0F3WUharjey6hG31pFxYwAa+Ofbjh8zHUD/P4DtCXHHdymzNRxm3Iz4oeqHC10+q2ZRlEmYlPVyiBPU5Xvc8Yd8Ib/B81IoSZlkkqfcVpGG77ZZ0vjCv2wBLlf64GCWbWlH0S+lqjTtMrZzFufrEB5eZ5sgS4yeMadK+i9oeRG50cd9+iHTZScouQ0vZ6KnZDGMUYvuVGK8o9UiKeNBZ9ffCSBcJ05cqSLkB62nNLsPIE09gfThOAjqhMsVhSMkiwxsk6ENFSLu9Zg/VNowui4taMSZFpdBfyHlVQU8ahO+G49119WD2OTQE4U0pjOYYT4ksqefpfylAXtw39jnj8auP2Y9LPHGiPN3xLEQMBIkC1FMpom22RWPCYoXhA8y+4435hy0hmkMuztYGxM1/BpedinAxsbqwvR1PJE7gfLFbCRLuVmtryQ+97jWMzpoAg///oDTXac1Yp8ie68/blmW//tdhOM9Q9zuh0viLyGgclbht1WLMBJ2eRF08H0Qh23D1AoudZmw3WtizuLKU40k25xHFu6V1VfDPt4UbuWuqPbwc44UUWIUgRM7f0Tk+7945dFyOVN0tHnvxHv+CJqyKJmLpxgLRnZINqsknvMt7MBO2vYxzvc4rW7ml+Ta2ECrXJYGPjClpp8M+vQpkS8+MALdrQki7er3Hx5uTnPqKCdiG0DdUFHWU/0Fb8JrLzZzEznPbmTp2dwy9YD0Xy4qlelqLev3WGZO6UZSxCSnCykqUfzH8c+gJ8+XU9QZOUY5WPcaEbyfxvl6eO+g+tflfw4gfUkfTkwSgftRsTVD2TDrfnYXn58v/HhVFbhE3SSgrfNoa1quiUqLzn4ui3iUDbFDKUDUkfqg2ZFDyaDGI6uQMTKa7PiB6WPzrX65SLAaSq9XeSiyzKVC8x33UsDmtIeEiXQNQ73pdXAiwfthht+UizryyDTK65IcnZuP3nwPQH4WBQSx/pCIhou02JrQsQKc+8ST1919DJJbVdFvUlMc2iurMpU6lDtsJOtFbu5qdjToTNe4CEouZA75BQXoaozbL5sEM2JXpqLjRLYXj5NMIK9u/tSf2wtYpptJAddbqgxPfxAEgRyaeXgj9BL8gDNvDL7Yqt1TP4Lx5SRBXMWYWwDy51YBf3ihszjNvNcqT5xIYvyQLDx37C9JyG9UNpZ92oMYfbAEFPftk072s9fneFtJwDeAgC5ucPWRJIsVPbbfZkY/VwqNGWwx7Hk6FfTWsL5nQOWFLxoatryPNfsb56c1ODw9Atn4H9bEIRvywPI8hDLZ8VyPYpuZ3Q5wonLidX3gF3Q+XijS76j1WdETsLG+pE/hw6b/pT/VGPTd0drNdIIo+VFb78kYuP7D8hOsCuK8onZjZ9nL+kEDUZuMlCAztla7ugkqJhwEpcL7tJfADltHrbdeCxi6v+0u1yvjFZWS3NL9a0ynnDOCQnXyAixInzAQjFqh14fyGXrPpvL9Uz5MGJ3WsdU4wfbPrkAX5J3GLEb3uA8vtzQdpLKtLq+mG+DkCgCA6o53GdeiR5i4h/LXpiuFpcP+ILBNzooqmW2EXsjUGvTzCkaT6ONXwdMnJDaHc677qrGAJvlbYeeXDXq/mq4VF54b9QF8dadS4NDURTOrL0pQEFhDtOmsNwVHPeqX",
    "__VIEWSTATEGENERATOR": "11E838D2",
    "__EVENTVALIDATION": "SWWB6bcdQIsBpEJd/aNpEz8Jgp1YTQjwwEAvZcBfX4kc5OTFL+8/WuTc7xRFe4EV/2M4xUABkKlDHkSCu9B1Kd+3uopmgSpGJQcV0tcYc8XwQch2STWConByaC3TMCH0+ZwszLPbgapJUOKBFL2sVYfkysplonAKlHIyZjIaJsBJiODjB/jatElPJWnCmEczbdGGGo9nPUkac7HnxyKtNgUOWr7fSkpXIlgRKjCpZXkTqfLaOWTFJn6HlAsGQs9dgeQnGd4rKwiKfvsOq2dKVzf1D4f8c3m6PcY/GPEg2S7NlOXAYQajt9QSFSYDc9bHDLWrXBXEx2YBwXhfb3JBWfa2lAQmmfEd54xl7sdxSu2YYo1TRU2xSD7IM1X7tx85yicvZj4aFrqPycrgz2vpnLgN1Cv177aM0sezObnhwmxaON65qlYCXZyBgdjnJUaYvqWnyhauCUhm4YXW13c0FpA1LE+gaWaqZidiXnFZhL2oeGZyVYvTrh9N8Scx9X7ady5QjYa9QM9QLgeNl6cqSmXXZyagkr1hh6zI9OXNW6o62iizyJu8du2cfe1Kt/B6bd48woBxYC8v6qY7DkfetYq5LVlrXyAHZ4K8A5WJFjaqyk7JBshAObAFRoA1CKFE2/M7iQ==",
    "forma": "請選擇遊戲",
    "Lotto649Control_history$txtNO": "",
    "Lotto649Control_history$chk": "radYM",
    "Lotto649Control_history$dropYear": 108,
    "Lotto649Control_history$dropMonth": 1,
    "Lotto649Control_history$btnSubmit": "查詢"
}


res = requests.post(url,data=payload,headers=headers)
soup = bs(res.text,"lxml")
# print(res.text)

In [45]:
### 解析 期別、開獎日、號碼
table_tag = soup.select("table.td_hm")[6]
# print(table_tag.select("tr"))

# 期別、開獎日 -> 第2個 tr
tr2 = table_tag.select("tr")[1]
lotto_no = tr2.select("td")[0].text
lotto_date = tr2.select("td")[1].text.strip()

# print(tr2)
print(lotto_no)
print(lotto_date)
print("-"*80)

# 號碼 -> 第5個 tr
tr5 = table_tag.select("tr")[4]
# print(tr5)

lotto_nums = []

for ele in tr5.select("td")[1:]:
    lotto_nums.append(ele.text.strip())
#     print(ele.text)


# List Comprehension 寫法
lotto_nums_short = [ ele.text.strip() for ele in tr5.select("td")[1:]]

print(lotto_nums)

print(lotto_nums_short)

108000003
108/01/08
--------------------------------------------------------------------------------
['04', '19', '21', '28', '39', '43', '18']
['04', '19', '21', '28', '39', '43', '18']


In [61]:
### List comprehension 
import random 
data = []

for i in range(1,7):
    data.append(random.randint(1,49))
    
print(data)

data2 = [ random.randint(1,49) for i in range(1,7)]

print(data2)

print([i**2 for i in range(1,11)])

[13, 31, 4, 33, 42, 34]
[9, 8, 18, 47, 34, 23]
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [64]:
### 整理成 functions

def get_clear_data(soup):
    
    result = []
    
    for table_tag in soup.select("table.td_hm"):
#         table_tag = soup.select("table.td_hm")[6]

        tr2 = table_tag.select("tr")[1]

        lotto_no = tr2.select("td")[0].text
        lotto_date = tr2.select("td")[1].text.strip()

        tr5 = table_tag.select("tr")[4]

        # List Comprehension 寫法
        lotto_nums_short = [ ele.text.strip() for ele in tr5.select("td")[1:]]

        data = {
            "no" : lotto_no,
            "time" : lotto_date,
            "numbers" : lotto_nums_short
        }
        
        result.append(data)
    
    return result
    

data = get_clear_data(soup)

data
# print(data)



[{'no': '108000009',
  'time': '108/01/29',
  'numbers': ['14', '20', '24', '30', '44', '48', '31']},
 {'no': '108000008',
  'time': '108/01/25',
  'numbers': ['03', '10', '14', '15', '20', '45', '02']},
 {'no': '108000007',
  'time': '108/01/22',
  'numbers': ['02', '20', '36', '37', '42', '46', '29']},
 {'no': '108000006',
  'time': '108/01/18',
  'numbers': ['04', '05', '10', '30', '39', '45', '14']},
 {'no': '108000005',
  'time': '108/01/15',
  'numbers': ['12', '20', '30', '38', '40', '41', '10']},
 {'no': '108000004',
  'time': '108/01/11',
  'numbers': ['13', '20', '22', '31', '34', '42', '09']},
 {'no': '108000003',
  'time': '108/01/08',
  'numbers': ['04', '19', '21', '28', '39', '43', '18']},
 {'no': '108000002',
  'time': '108/01/04',
  'numbers': ['06', '07', '08', '20', '47', '48', '30']},
 {'no': '108000001',
  'time': '108/01/01',
  'numbers': ['18', '23', '24', '25', '39', '42', '48']}]

In [69]:
### 解析 POST payload 隱藏參數 
def parsing_hidden_params(soup):

    # __VIEWSTATEGENERATOR

    __VIEWSTATEGENERATOR = soup.select("input#__VIEWSTATEGENERATOR")[0]["value"]

    # __EVENTVALIDATION
    __EVENTVALIDATION = soup.select("input#__EVENTVALIDATION")[0]["value"]

    # __VIEWSTATE
    __VIEWSTATE = soup.select("input#__VIEWSTATE")[0]["value"]

    hidden = {
        "__VIEWSTATEGENERATOR" : __VIEWSTATEGENERATOR,
        "__EVENTVALIDATION" : __EVENTVALIDATION,
        "__VIEWSTATE" : __VIEWSTATE,
    }
    
    return hidden

p = parsing_hidden_params(soup)
print(p)

{'__VIEWSTATEGENERATOR': '11E838D2', '__EVENTVALIDATION': 'Z0mTflLUWbcEOqJgcqLjUX8NbE49Rvv//oSu9SfGIottK9S/8EiDRhZFhhtTr2UO1fvQfu8gNTrxJ1mmRk5OCJ7NkHo4yw/cA3KkzJBzEYEj3fv/YaxeL5Y3Jc/1IqP22rLx4rGjRJIE3zKF91IuBvj5Jcu/KQXXn2yKsIVDsbNCjwrEXa2zgtPD4RQRc7ECp5SW7Zi3CI7GNba/bxznOZYXxEVF8ziFByV3wIctr6QJp7cMrp5ylI89dvZfvILx93WeXhmPvl1YIXwE7EJqkQC18VKg9fO1ZgcGAy3PUcBxOHn+eWaEjOtVygACS7DOw7TrdWaHYI87KNsbxL4EPxSRj0qGQvUzQbX8ArO2AllvoFnvRyul86JLXUBheA9BOrR6MiiZcQ0xxcvxmBzLC6zV38+G5RY4BWI8tHoFvIa2aF6I7fmKcB2s2xjVGQT693ANjVRa47qvJKLkrZRc50rbdK0qtAyC/NI7VUz5cAs3bIt784lLcUujSzIrc/vBBPLB+shqsohljRCD3Mo8J94ZtffLSPsBHfQwftZLtYCWGh24ymklWCKk95GPcSRft3vMj9vir518eRutqZaUYojeguw1ZqmtKY4+FFAkjqBCfd+lIxYrfGimBrasCl05M/bwGw==', '__VIEWSTATE': 'oHy7SPUnc8CjVr4zWozIGVQqRMY7yKC3367Bf2dhC/NwjoI/fmv3CHpaGa/K8ZthavRcolh4rpHCWL9IjLwj4n4G6pvi3gNNLNWnz1HkTWNApotZMLykGuVJBpqo2Pw83k3wUUzeQ8DnVyM0fHolDtqGi5AnVtpvJhggv552h/+25sPy6dsiMj59Ssotj88Z2yiizDg2HxW829m+UWg1Ypq+HUlkBzEmi0Hfo6omaYZI6adnDvMFeBUo9NlkqECcaf/pdhE6JyJ5sX409d

In [72]:
### 抓取 1 ~ 6 月資料
results = []

url = "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx"
headers = {
    "Host"      : "www.taiwanlottery.com.tw",
    "Origin"    : "http://www.taiwanlottery.com.tw",
    "Referer"   : "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

for i in range(1,7):
    
    hidden = parsing_hidden_params(soup)
    
    payload = {
        "__VIEWSTATE": hidden["__VIEWSTATE"],
        "__VIEWSTATEGENERATOR": hidden["__VIEWSTATEGENERATOR"],
        "__EVENTVALIDATION": hidden["__EVENTVALIDATION"],
        "forma": "請選擇遊戲",
        "Lotto649Control_history$txtNO": "",
        "Lotto649Control_history$chk": "radYM",
        "Lotto649Control_history$dropYear": 108,
        "Lotto649Control_history$dropMonth": i,
        "Lotto649Control_history$btnSubmit": "查詢"
    }

    res = requests.post(url,data=payload,headers=headers)
    soup = bs(res.text,"lxml")

    data = get_clear_data(soup)
    
    results += data

print("Done.")

Done.


In [76]:
sorted(results,key=lambda ele : ele["no"])

[{'no': '108000001',
  'time': '108/01/01',
  'numbers': ['18', '23', '24', '25', '39', '42', '48']},
 {'no': '108000002',
  'time': '108/01/04',
  'numbers': ['06', '07', '08', '20', '47', '48', '30']},
 {'no': '108000003',
  'time': '108/01/08',
  'numbers': ['04', '19', '21', '28', '39', '43', '18']},
 {'no': '108000004',
  'time': '108/01/11',
  'numbers': ['13', '20', '22', '31', '34', '42', '09']},
 {'no': '108000005',
  'time': '108/01/15',
  'numbers': ['12', '20', '30', '38', '40', '41', '10']},
 {'no': '108000006',
  'time': '108/01/18',
  'numbers': ['04', '05', '10', '30', '39', '45', '14']},
 {'no': '108000007',
  'time': '108/01/22',
  'numbers': ['02', '20', '36', '37', '42', '46', '29']},
 {'no': '108000008',
  'time': '108/01/25',
  'numbers': ['03', '10', '14', '15', '20', '45', '02']},
 {'no': '108000009',
  'time': '108/01/29',
  'numbers': ['14', '20', '24', '30', '44', '48', '31']},
 {'no': '108000010',
  'time': '108/02/01',
  'numbers': ['10', '17', '20', '21', 

In [81]:
#### 組合程式
import requests 
from bs4 import BeautifulSoup as bs

def get_clear_data(soup):
    
    result = []
    
    for table_tag in soup.select("table.td_hm"):

        tr2 = table_tag.select("tr")[1]

        lotto_no = tr2.select("td")[0].text
        lotto_date = tr2.select("td")[1].text.strip()

        tr5 = table_tag.select("tr")[4]

        # List Comprehension 寫法
        lotto_nums_short = [ ele.text.strip() for ele in tr5.select("td")[1:]]

        data = {
            "no" : lotto_no,
            "time" : lotto_date,
            "numbers" : lotto_nums_short
        }
        
        result.append(data)
    
    return result

def parsing_hidden_params(soup):

    __VIEWSTATEGENERATOR = soup.select("input#__VIEWSTATEGENERATOR")[0]["value"]
    __EVENTVALIDATION    = soup.select("input#__EVENTVALIDATION")[0]["value"]
    __VIEWSTATE          = soup.select("input#__VIEWSTATE")[0]["value"]

    return {
        "__VIEWSTATEGENERATOR" : __VIEWSTATEGENERATOR,
        "__EVENTVALIDATION" : __EVENTVALIDATION,
        "__VIEWSTATE" : __VIEWSTATE,
    }




### 主程式 (main)

# 1. 發 GET requests , 取得第一次的 hidden params
url = "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx"
headers = {
    "Host"      : "www.taiwanlottery.com.tw",
    "Origin"    : "http://www.taiwanlottery.com.tw",
    "Referer"   : "http://www.taiwanlottery.com.tw/Lotto/Lotto649/history.aspx",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res1 = requests.get(url,headers=headers)
soup1 = bs(res1.text,"lxml")
hidden = parsing_hidden_params(soup1)

# 2. for-loop 抓取 1 ~ 6 月資料

dataList = []

for i in range(1,7):
    payload = {
        "__VIEWSTATE": hidden["__VIEWSTATE"],
        "__VIEWSTATEGENERATOR": hidden["__VIEWSTATEGENERATOR"],
        "__EVENTVALIDATION": hidden["__EVENTVALIDATION"],
        "forma": "請選擇遊戲",
        "Lotto649Control_history$txtNO": "",
        "Lotto649Control_history$chk": "radYM",
        "Lotto649Control_history$dropYear": 108,
        "Lotto649Control_history$dropMonth": i,
        "Lotto649Control_history$btnSubmit": "查詢"
    }

    res2 = requests.post(url,data=payload,headers=headers)
    soup2 = bs(res2.text,"lxml")
    
    data = get_clear_data(soup2)
    
    dataList += data
    
    hidden = parsing_hidden_params(soup2)
    
    print("{} 月 is ok.".format(i))

print("Done.")

1 月 is ok.
2 月 is ok.
3 月 is ok.
4 月 is ok.
5 月 is ok.
6 月 is ok.
Done.


In [83]:
sorted(dataList,key = lambda ele: ele["no"])

[{'no': '108000001',
  'time': '108/01/01',
  'numbers': ['18', '23', '24', '25', '39', '42', '48']},
 {'no': '108000002',
  'time': '108/01/04',
  'numbers': ['06', '07', '08', '20', '47', '48', '30']},
 {'no': '108000003',
  'time': '108/01/08',
  'numbers': ['04', '19', '21', '28', '39', '43', '18']},
 {'no': '108000004',
  'time': '108/01/11',
  'numbers': ['13', '20', '22', '31', '34', '42', '09']},
 {'no': '108000005',
  'time': '108/01/15',
  'numbers': ['12', '20', '30', '38', '40', '41', '10']},
 {'no': '108000006',
  'time': '108/01/18',
  'numbers': ['04', '05', '10', '30', '39', '45', '14']},
 {'no': '108000007',
  'time': '108/01/22',
  'numbers': ['02', '20', '36', '37', '42', '46', '29']},
 {'no': '108000008',
  'time': '108/01/25',
  'numbers': ['03', '10', '14', '15', '20', '45', '02']},
 {'no': '108000009',
  'time': '108/01/29',
  'numbers': ['14', '20', '24', '30', '44', '48', '31']},
 {'no': '108000010',
  'time': '108/02/01',
  'numbers': ['10', '17', '20', '21', 